In [1]:
import json
import os
import pandas as pd
import zipfile

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
input_folder = "privacy_scraper/blacklight_json"

In [26]:
def process_json_files(input_folder: str) -> list:
    """
    Process JSON files to extract tracking metrics from DuckDuckGo privacy data.
    
    Args:
        input_folder (str): Path to folder containing JSON files
        
    Returns:
        list: List of dictionaries containing extracted metrics
    """
    rows = []
    
    for filename in os.listdir(input_folder):
        if not filename.endswith(".json"):
            continue
            
        file_path = os.path.join(input_folder, filename)
        
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
            
            domain_name = filename.replace(".json", "")
            cards = data.get("groups", [])[0].get("cards", [])
            
            metrics = {
                "filename": domain_name,
                "ddg_join_ads": 0,
                "third_party_cookies": 0,
                "canvas_fingerprinting": 0,
                "session_recording": 0,
                "key_logging": 0,
                "fb_pixel": 0,
                "google_analytics": 0
            }
            
            for card in cards:
                card_type = card.get("cardType", "")
                if card_type == "ddg_join_ads":
                    metrics["ddg_join_ads"] = card.get("bigNumber", 0)
                elif card_type == "cookies":
                    metrics["third_party_cookies"] = card.get("bigNumber", 0)
                elif card_type in ["canvas_fingerprinters", "session_recorders", 
                                 "key_logging", "fb_pixel_events"]:
                    metric_key = {
                        "canvas_fingerprinters": "canvas_fingerprinting",
                        "session_recorders": "session_recording",
                        "key_logging": "key_logging",
                        "fb_pixel_events": "fb_pixel"
                    }[card_type]
                    metrics[metric_key] = 1 if card.get("testEventsFound", False) else 0
                elif card_type == "ga":
                    metrics["google_analytics"] = 1 if card.get("testEventsFound", False) else 0
            
            rows.append(metrics)
        
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            
    return rows

In [6]:
govdir_blacklight_df = pd.DataFrame(process_json_files("privacy_scraper/blacklight_json"))
govdir_blacklight_df.shape

(3877, 8)

In [7]:
govdir_blacklight_df.head()

,filename,ddg_join_ads,third_party_cookies,canvas_fingerprinting,session_recording,key_logging,fb_pixel,google_analytics
0,www.freienbach.ch_,0,0,0,0,1,0,0
1,drangsnes.is_,0,0,0,0,0,0,0
2,www.wald.zh.ch_,0,0,0,0,1,0,0
3,www.bournens.ch_,1,0,0,0,0,0,0
4,hagneck.ch_,3,0,0,0,0,0,0


In [8]:
govdir_blacklight_df.to_csv("../data/blacklight_domain.csv", index = False)

In [9]:
govdir_blacklight_df.describe()

,ddg_join_ads,third_party_cookies,canvas_fingerprinting,session_recording,key_logging,fb_pixel,google_analytics
count,3877.000000,3877.000000,3877.000000,3877.000000,3877.000000,3877.000000,3877.000000
mean,0.388703,0.207893,0.004127,0.004643,0.084859,0.006706,0.000774
std,0.854362,0.989683,0.064117,0.067988,0.278708,0.081627,0.027810
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8.000000,15.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
govdir = pd.read_csv("../data/combined_tsv_data.tsv", sep = "\t")
govdir.head()

,Name,Govdirectory URL,Type,Website,source_file
0,Cherkasy Oblast,https://www.govdirectory.org/ukraine/Q161808/,oblast of Ukraine,https://www.oblradack.gov.ua/,Ukraine
1,Chernihiv Oblast,https://www.govdirectory.org/ukraine/Q167874/,oblast of Ukraine,https://cg.gov.ua/,Ukraine
2,Chernivtsi Oblast,https://www.govdirectory.org/ukraine/Q168856/,oblast of Ukraine,https://bukoda.gov.ua/,Ukraine
3,Dnipropetrovsk Oblast,https://www.govdirectory.org/ukraine/Q170672/,oblast of Ukraine,http://www.adm.dp.gov.ua/,Ukraine
4,Donetsk Oblast,https://www.govdirectory.org/ukraine/Q2012050/,oblast of Ukraine,https://dn.gov.ua/,Ukraine


In [11]:
# Keys
govdir_blacklight_df['filename'] = govdir_blacklight_df['filename'].str.rstrip('_')
govdir['Website'] = govdir['Website'].str.replace(r'^https?://', '', regex=True)
govdir['Website'] = govdir['Website'].str.rstrip('/')

In [12]:
fin_df = govdir.merge(govdir_blacklight_df, 
            left_on = "Website",
            right_on = "filename",
                     how = "left")

In [13]:
fin_df.shape

(12255, 13)

In [14]:
fin_df.head()

,Name,Govdirectory URL,Type,Website,source_file,filename,ddg_join_ads,third_party_cookies,canvas_fingerprinting,session_recording,key_logging,fb_pixel,google_analytics
0,Cherkasy Oblast,https://www.govdirectory.org/ukraine/Q161808/,oblast of Ukraine,www.oblradack.gov.ua,Ukraine,www.oblradack.gov.ua,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Chernihiv Oblast,https://www.govdirectory.org/ukraine/Q167874/,oblast of Ukraine,cg.gov.ua,Ukraine,cg.gov.ua,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Chernivtsi Oblast,https://www.govdirectory.org/ukraine/Q168856/,oblast of Ukraine,bukoda.gov.ua,Ukraine,bukoda.gov.ua,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Dnipropetrovsk Oblast,https://www.govdirectory.org/ukraine/Q170672/,oblast of Ukraine,www.adm.dp.gov.ua,Ukraine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Donetsk Oblast,https://www.govdirectory.org/ukraine/Q2012050/,oblast of Ukraine,dn.gov.ua,Ukraine,dn.gov.ua,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
fin_df.groupby('source_file').describe().reset_index().reset_index().dropna()

index  source_file ddg_join_ads                                          \
                             count      mean       std  min  25%  50%  75%   
0      0      Austria        389.0  0.154242  0.539056  0.0  0.0  0.0  0.0   
1      1      Belgium        444.0  0.423423  0.931511  0.0  0.0  0.0  1.0   
2      2      Bermuda          2.0  3.000000  1.414214  2.0  2.5  3.0  3.5   
3      3     Cameroon         14.0  0.214286  0.425815  0.0  0.0  0.0  0.0   
7      7   East Timor          6.0  0.000000  0.000000  0.0  0.0  0.0  0.0   
10    10        Ghana         51.0  0.686275  1.378263  0.0  0.0  0.0  1.0   
11    11      Iceland         59.0  1.016949  1.370773  0.0  0.0  1.0  1.0   
15    15       Latvia         18.0  0.333333  0.685994  0.0  0.0  0.0  0.0   
17    17     Malaysia         29.0  0.896552  1.113066  0.0  0.0  1.0  1.0   
20    20        Nepal         88.0  1.670455  1.467828  0.0  0.0  1.0  3.0   
27    27       Russia          9.0  0.444444  0.527046  0.0  0.0  0.0  1.0   
30    30       Sweden        490.0  0.214286  0.671506  0.0  0.0  0.0  0.0   
31    31  Switzerland       1981.0  0.323069  0.746451  0.0  0.0  0.0  0.0   
32    32      Ukraine         27.0  1.481481  1.451004  0.0  1.0  1.0  2.0   

        third_party_cookies                                                \
    max               count      mean       std  min  25%  50%  75%   max   
0   4.0               389.0  0.169666  0.980219  0.0  0.0  0.0  0.0  14.0   
1   7.0               444.0  0.189189  1.085621  0.0  0.0  0.0  0.0  12.0   
2   4.0                 2.0  0.000000  0.000000  0.0  0.0  0.0  0.0   0.0   
3   1.0                14.0  0.285714  1.069045  0.0  0.0  0.0  0.0   4.0   
7   0.0                 6.0  0.000000  0.000000  0.0  0.0  0.0  0.0   0.0   
10  6.0                51.0  0.333333  1.608312  0.0  0.0  0.0  0.0  11.0   
11  5.0                59.0  0.406780  1.131350  0.0  0.0  0.0  0.0   5.0   
15  2.0                18.0  0.333333  0.485071  0.0  0.0  0.0  1.0   1.0   
17  4.0                29.0  0.724138  1.810615  0.0  0.0  0.0  0.0   8.0   
20  4.0                88.0  0.545455  1.923294  0.0  0.0  0.0  0.0   8.0   
27  1.0                 9.0  0.666667  1.322876  0.0  0.0  0.0  1.0   4.0   
30  7.0               490.0  0.206122  0.980570  0.0  0.0  0.0  0.0  15.0   
31  8.0              1981.0  0.167592  0.858810  0.0  0.0  0.0  0.0  11.0   
32  5.0                27.0  0.444444  1.339728  0.0  0.0  0.0  0.0   5.0   

   canvas_fingerprinting                                               \
                   count      mean       std  min  25%  50%  75%  max   
0                  389.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
1                  444.0  0.009009  0.094594  0.0  0.0  0.0  0.0  1.0   
2                    2.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
3                   14.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
7                    6.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
10                  51.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
11                  59.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
15                  18.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
17                  29.0  0.034483  0.185695  0.0  0.0  0.0  0.0  1.0   
20                  88.0  0.011364  0.106600  0.0  0.0  0.0  0.0  1.0   
27                   9.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
30                 490.0  0.006122  0.078086  0.0  0.0  0.0  0.0  1.0   
31                1981.0  0.002524  0.050188  0.0  0.0  0.0  0.0  1.0   
32                  27.0  0.037037  0.192450  0.0  0.0  0.0  0.0  1.0   

   session_recording                                              key_logging  \
               count      mean       std  min  25%  50%  75%  max       count   
0              389.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0       389.0   
1              444.0  0.013514  0.115590  0.0  0.0  0.0  0.0  1.0       444.0   
2                2.0  0.0000

In [16]:
fin_df.groupby(['source_file', 'Type']).describe().reset_index().reset_index().dropna()

index  source_file                                       Type  \
                                                                    
0       0      Austria                          Bundesministerium   
1       1      Austria                        district of Austria   
2       2      Austria                   federal state of Austria   
3       3      Austria                    municipality of Austria   
4       4      Belgium                        Belgian delegations   
6       6      Belgium                     Federal Public Service   
7       7      Belgium               Federal Scientific Institute   
8       8      Belgium      Public Institution of Social Security   
9       9      Belgium              Public Interest Organizations   
10     10      Belgium                    Public Planning Service   
16     16      Belgium                                    embassy   
17     17      Belgium                             emergency zone   
20     20      Belgium                    municipality of Belgium   
21     21      Belgium                                police zone   
22     22      Belgium                        province of Belgium   
30     30     Cameroon                       ministry of Cameroon   
44     44   East Timor                    ministry of Timor-Leste   
60     60        Ghana                          Ministry of Ghana   
61     61        Ghana                          district of Ghana   
62     62        Ghana                            region of Ghana   
66     66      Iceland                    municipality of Iceland   
77     77       Latvia                         Ministry of Latvia   
80     80       Latvia                     municipality of Latvia   
85     85     Malaysia                       district of Malaysia   
86     86     Malaysia              federal territory of Malaysia   
87     87     Malaysia                       ministry of Malaysia   
88     88     Malaysia                          state of Malaysia   
97     97        Nepal                          district of Nepal   
98     98        Nepal                          ministry of Nepal   
99     99        Nepal                          province of Nepal   
138   138       Russia                 federal ministry in Russia   
152   152       Sweden                            AP pension fund   
156   156       Sweden                  Swedish government agency   
157   157       Sweden  administrative courts of appeal in Sweden   
158   158       Sweden                authority under the Riksdag   
159   159       Sweden                            court of appeal   
161   161       Sweden                             district court   
164   164       Sweden                     municipality of Sweden   
165   165       Sweden                             public utility   
166   166       Sweden                 regional council in Sweden   
169   169  Switzerland                      canton of Switzerland   
170   170  Switzerland          federal department of Switzerland   
171   171  Switzerland                municipality of Switzerland   
172   172      Ukraine                         Ukrainian ministry   
174   174      Ukraine                          oblast of Ukraine   

    ddg_join_ads                                                 \
           count      mean       std  min   25%  50%   75%  max   
0            8.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0   
1            2.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0   
2            7.0  0.428571  1.133893  0.0  0.00  0.0  0.00  3.0   
3          372.0  0.153226  0.530133  0.0  0.00  0.0  0.00  4.0   
4            6.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0   
6            4.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0   
7            6.0  1.833333  2.041241  0.0  0.00  1.5  3.75  4.0   
8            5.0  0.200000  0.447214  0.0  0.00  0.0  0.00  1.0   
9            7.0  1.571429  1.812654  0.0  0.50  1.0  2.00  5.0   
10           2.0  0.500

### Let's do the same for US gov. list

In [27]:
usgov_blacklight_df = pd.DataFrame(process_json_files("privacy_scraper/us_blacklight_json/"))
usgov_blacklight_df.shape

Error processing merchantvillenj.gov.json: list index out of range
Error processing unionbeachnj.gov.json: list index out of range
Error processing ibab.gov.json: list index out of range
Error processing westfieldnj.gov.json: list index out of range
Error processing clackesd.gov.json: list index out of range
Error processing alertoc.gov.json: list index out of range
Error processing nmijudiciary.gov.json: list index out of range
Error processing navajocountysheriff.gov.json: list index out of range
Error processing nohatenj.gov.json: list index out of range
Error processing jwod.gov.json: list index out of range
Error processing villageoflindenhurstny.gov.json: list index out of range
Error processing townofclymanwi.gov.json: list index out of range
Error processing townofwinchesterwi.gov.json: list index out of range
Error processing williamstownma.gov.json: list index out of range
Error processing birthcontrol.gov.json: list index out of range
Error processing mnhousing.gov.json: lis

Error processing floridadep.gov.json: list index out of range
Error processing tullahomatn.gov.json: list index out of range
Error processing beoutsideidaho.gov.json: list index out of range
Error processing history.gov.json: list index out of range
Error processing coda18.gov.json: list index out of range
Error processing circuitclerkmarioncountyms.gov.json: list index out of range
Error processing bosque.gov.json: list index out of range
Error processing ucr.gov.json: list index out of range
Error processing westhamptonbeach.gov.json: list index out of range
Error processing newberrycounty.gov.json: list index out of range
Error processing sandimasca.gov.json: list index out of range
Error processing cloud.gov.json: list index out of range
Error processing firstnetme.gov.json: list index out of range
Error processing puliklatribe.gov.json: list index out of range
Error processing pclob.gov.json: list index out of range
Error processing nclawspecialists.gov.json: list index out of ran

Error processing uscis.gov.json: list index out of range
Error processing morgancocourtsoh.gov.json: list index out of range
Error processing gearupiowa.gov.json: list index out of range
Error processing youngsvillenc.gov.json: list index out of range
Error processing mi.gov.json: list index out of range
Error processing bankhelp.gov.json: list index out of range
Error processing nlrb.gov.json: list index out of range
Error processing earmarks.gov.json: list index out of range
Error processing nutrition.gov.json: list index out of range
Error processing wakeforestnc.gov.json: list index out of range
Error processing yellcountyar.gov.json: list index out of range
Error processing townofwilkeson.gov.json: list index out of range
Error processing bridgeportct.gov.json: list index out of range
Error processing townofwalworthny.gov.json: list index out of range
Error processing readync.gov.json: list index out of range
Error processing poloil.gov.json: list index out of range
Error processi

Error processing senatecalendar.gov.json: list index out of range
Error processing sandag.gov.json: list index out of range
Error processing woodruffcountyar.gov.json: list index out of range
Error processing cityoflfp.gov.json: list index out of range
Error processing userra.gov.json: list index out of range
Error processing southtucsonaz.gov.json: list index out of range
Error processing middletownct.gov.json: list index out of range
Error processing pueblocounty.gov.json: list index out of range
Error processing wartimecontracting.gov.json: list index out of range
Error processing warrentonga.gov.json: list index out of range
Error processing nvcourts.gov.json: list index out of range
Error processing fjc.gov.json: list index out of range
Error processing ustreas.gov.json: list index out of range
Error processing cartertnsheriff.gov.json: list index out of range
Error processing cityofcanalfulton-oh.gov.json: list index out of range
Error processing somervillema.gov.json: list index

Error processing westyorkpa.gov.json: list index out of range
Error processing frc.gov.json: list index out of range
Error processing townoftruckee.gov.json: list index out of range
Error processing colfaxtwpmecosta.gov.json: list index out of range
Error processing soboba-nsn.gov.json: list index out of range
Error processing presidentialinnovationfellows.gov.json: list index out of range
Error processing turtlelaketwpcassco.gov.json: list index out of range
Error processing cattco.gov.json: list index out of range
Error processing floodsmart.gov.json: list index out of range
Error processing ypsitownship.gov.json: list index out of range
Error processing townofkingwi.gov.json: list index out of range
Error processing yorktownny.gov.json: list index out of range
Error processing wvunclaimedproperty.gov.json: list index out of range
Error processing minnesota.gov.json: list index out of range
Error processing thurstoncountywa.gov.json: list index out of range
Error processing surrycoun

...

Error processing mvdmt.gov.json: list index out of range
Error processing everykidoutdoors.gov.json: list index out of range
Error processing juntos.gov.json: list index out of range
Error processing paslc.gov.json: list index out of range
Error processing guideline.gov.json: list index out of range
Error processing redbay-al.gov.json: list index out of range
Error processing mha.gov.json: list index out of range
Error processing vcpa.gov.json: list index out of range
Error processing santaana-nsn.gov.json: list index out of range
Error processing westlaketx.gov.json: list index out of range
Error processing nbwaor.gov.json: list index out of range
Error processing mwdsls.gov.json: list index out of range
Error processing wilderky.gov.json: list index out of range
Error processing kansasblind.gov.json: list index out of range
Error processing 9-11commission.gov.json: list index out of range
Error processing asap.gov.json: list index out of range
Error processing californiacity-ca.gov.j

Error processing wisecountyva.gov.json: list index out of range
Error processing bush41library.gov.json: list index out of range
Error processing cordovarpd.gov.json: list index out of range
Error processing alleganycountysheriff-ny.gov.json: list index out of range
Error processing energycodes.gov.json: list index out of range
Error processing fdicoig.gov.json: list index out of range
Error processing hendersonvillenc.gov.json: list index out of range
Error processing irsauctions.gov.json: list index out of range
Error processing cachesheriff.gov.json: list index out of range
Error processing cbp.gov.json: list index out of range
Error processing copperopolisfire.gov.json: list index out of range
Error processing townofmonroewi.gov.json: list index out of range
Error processing jordanrivercommission.gov.json: list index out of range
Error processing nsopw.gov.json: list index out of range
Error processing rushriverwi.gov.json: list index out of range
Error processing ice.gov.json: lis

Error processing republicanwhip.gov.json: list index out of range
Error processing itdashboard.gov.json: list index out of range
Error processing stluciesheriff.gov.json: list index out of range
Error processing southwebsterohio.gov.json: list index out of range
Error processing azwifa.gov.json: list index out of range
Error processing ofallonmo.gov.json: list index out of range
Error processing carrollcountyva.gov.json: list index out of range
Error processing bis.gov.json: list index out of range
Error processing carrollcountyil.gov.json: list index out of range
Error processing steelecountynd.gov.json: list index out of range
Error processing ndhan.gov.json: list index out of range
Error processing pbcpao.gov.json: list index out of range
Error processing villageofgouverneurny.gov.json: list index out of range
Error processing ncirc.gov.json: list index out of range
Error processing voa.gov.json: list index out of range
Error processing vernonia-or.gov.json: list index out of range


Error processing govloans.gov.json: list index out of range
Error processing warrencountypa.gov.json: list index out of range
Error processing findtreatment.gov.json: list index out of range
Error processing vbhil.gov.json: list index out of range
Error processing cosumnescsd.gov.json: list index out of range
Error processing pcpao.gov.json: list index out of range
Error processing digital.gov.json: list index out of range
Error processing kansastag.gov.json: list index out of range
Error processing chautauquacountyks.gov.json: list index out of range
Error processing mymedicare.gov.json: list index out of range
Error processing rvss-or.gov.json: list index out of range
Error processing womenshealth.gov.json: list index out of range
Error processing doi.gov.json: list index out of range
Error processing osrdmo.gov.json: list index out of range
Error processing unifiedfireut.gov.json: list index out of range
Error processing darrouzettisd.gov.json: list index out of range
Error processi

(6251, 8)

In [28]:
usgov_blacklight_df.head()

,filename,ddg_join_ads,third_party_cookies,canvas_fingerprinting,session_recording,key_logging,fb_pixel,google_analytics
0,vermontvilletownshipmi.gov,1,1,0,0,0,0,0
1,snoqualmiewa.gov,5,0,0,0,0,0,0
2,bucklinmo.gov,0,5,0,0,0,0,0
3,bentoncountywa.gov,0,0,0,0,0,0,0
4,stephenscity.gov,0,0,0,0,0,0,0


In [29]:
usgov_blacklight_df.describe()

,ddg_join_ads,third_party_cookies,canvas_fingerprinting,session_recording,key_logging,fb_pixel,google_analytics
count,6251.000000,6251.000000,6251.000000,6251.000000,6251.000000,6251.000000,6251.000000
mean,1.683571,1.108463,0.012958,0.014238,0.018077,0.026876,0.005599
std,2.780679,4.342911,0.113102,0.118479,0.133241,0.161733,0.074623
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,37.000000,86.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Let's do it for Indian Gov. List

In [20]:
ingov_blacklight_df = pd.DataFrame(process_json_files("privacy_scraper/in_blacklight_json"))
ingov_blacklight_df.shape

FileNotFoundError: [Errno 2] No such file or directory: 'privacy_scraper/in_blacklight_json'